# Lasso Scenario Creation Quickstart

In this notebook we will run through:

1. Using a configuration file to run lasso  
2. Setting up a base sc
enario and applying projects  
3. Transforming the standard network format to the MetCouncil expected format    
4. Exporting the network to a shapefile and csvs 

In [ ]:
import os
import sys
import yaml

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit

In [10]:
from lasso import CubeTransit

ImportError: cannot import name 'ModelToStdAdapter' from 'lasso.model_transit' (/Users/elizabeth/Documents/urbanlabs/MetCouncil/working/Lasso/lasso/model_transit.py)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
# if you don't want to see so much detail, set to logging.INFO or DEBUG
logger.setLevel(logging.DEBUG)


## 1 - Set Base Roadway and Transit Network Locations

In [ ]:
# Examples set here for testing purposes
EXAMPLE_ROADWAY_DIRECTORY = os.path.join(
    os.path.dirname(os.path.abspath('')), "examples", "stpaul"
)

EXAMPLE_TRANSIT_DIRECTORY = os.path.join(
    os.path.dirname(os.path.abspath('')), "examples", "stpaul"
)

In [ ]:
# set this to the path where your roadway files are  i.e. r"c:\networks\myroad\base"
BASE_ROADWAY_DIRECTORY = EXAMPLE_ROADWAY_DIRECTORY

# set this to the path where your transit files are i.e. r"c:\networks\mytransit\base"
BASE_TRANSIT_DIRECTORY = EXAMPLE_TRANSIT_DIRECTORY

## 2 - Scenario Settings

In [ ]:
# Examples set here for testing purposes
EXAMPLE_PROJECT_CARD_DIRECTORY = os.path.join(
    os.path.dirname(os.path.abspath('')), "examples", "stpaul", "project_cards"
)

EXAMPLE_CONFIG_FILE = os.path.join(
    os.path.dirname(os.path.abspath('')), "examples", "settings", "my_config.yaml"
)

EXAMPLE_OUTPUT_DIRECTORY = os.path.join(
    os.path.dirname(os.path.abspath('')), "tests", "scratch"
)

In [ ]:
# set this to the path of the folder where the project cards are i.e. r"C:\networks\project_cards"
PROJECT_CARD_DIRECTORY = EXAMPLE_PROJECT_CARD_DIRECTORY

# set this to the path of the config file you want to use i.e. r"C:\networks\configs\base_config.yml"
MY_CONFIG_FILE = EXAMPLE_CONFIG_FILE  

# set this to the path of the folder you want output to b e i.e. r"C:\networks\built_nets\base"
OUTPUT_DIRECTORY = EXAMPLE_OUTPUT_DIRECTORY

## 3 - Create Scenario

### Base Scenario: what networks are you adding the projects to?

Base scenarios must at the least specify a highway network but can also specify a directory where transit networks can be found.  

In this step the highway and transit networks are read in and validated to each other – making sure that transit nodes exist and the routes can be followed along the roadway network..  

Validation can take some time, so if you are already confident that your roadway and transit networks are valid and compatible you may want to override the validation using the flag: `validate = False`.

In [ ]:
base_scenario = Scenario.create_base_scenario(
        "shape.geojson",
        "link.json",
        "node.geojson",
        roadway_dir=BASE_ROADWAY_DIRECTORY,
        transit_dir=BASE_TRANSIT_DIRECTORY,
        validate=False
    )

### Scenario Creation: adding projects to the base scenario

As coded, this will add projects from the `PROJECT_CARD_DIRECTORY` and if specified here, will filter based on a glob search or a tag.

In [ ]:
my_scenario=None

my_scenario = Scenario.create_scenario(
    base_scenario=base_scenario,
    card_directory=PROJECT_CARD_DIRECTORY,
    #tags=,   #uncomment if you want to use
    #glob_search=,  #uncomment if you want to use
    validate_project_cards=False,
)

### Standard Networks --> Model Networks

At this point, we need to calculate all the variables into what MetCouncil's model is expecting. The method `roadway_standard_to_met_council_network()` broadly does the following:  
 
- creates a parallel managed lane network
- calculates additional variables based on geography or other variables (i.e. county, assignment group, area type, etc)
- flattens variables stored as continuous time values and determines their value by time period (i.e. lanes_am)   
- reprojects into MetCouncil's projection

In [ ]:
if MY_CONFIG_FILE:
    with open(MY_CONFIG_FILE) as f:
        my_config = yaml.safe_load(f)
else:
    my_config = {}

model_road_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    my_scenario.road_net, parameters=my_config.get("my_parameters", {})
)
model_road_net.roadway_standard_to_met_council_network()

In [ ]:
standard_transit = StandardTransit.fromTransitNetwork(my_scenario.transit_net)

## 4 -  Export Networks

### Roadway Model Network --> Shapefiles

In [ ]:
model_road_net.write_roadway_as_shp(
    output_link_shp = "links.shp",
    output_node_shp = "nodes.shp",
)

###  Roadway Model Network --> Fixed Width for Cube

In [ ]:
model_road_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(OUTPUT_DIRECTORY,"links.txt"),
    output_node_txt = os.path.join(OUTPUT_DIRECTORY,"nodes.txt"),
    output_link_header_width_txt = os.path.join(OUTPUT_DIRECTORY,"links_header_width.txt"),
    output_node_header_width_txt = os.path.join(OUTPUT_DIRECTORY,"nodes_header_width.txt"),
    output_cube_network_script = os.path.join(OUTPUT_DIRECTORY,"make_complete_network_from_fixed_width_file.s"),
)

### Transit Networks --> Cube Lin File

In [ ]:
standard_transit.write_as_cube_lin(outpath = os.path.join(OUTPUT_DIRECTORY,"transit.lin") )